# Assignment 1
Bradley Thompson - CS 510 LLM Winter 2024

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")

/Users/bradleythompson/Projects/python/llm-pdx/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
tokenizer_config.json: 100%|████████████████████| 222/222 [00:00<00:00, 289kB/s]
tokenizer.json: 100%|██████████████████████| 14.5M/14.5M [00:00<00:00, 42.3MB/s]
special_tokens_map.json: 100%|████████████████| 85.0/85.0 [00:00<00:00, 187kB/s]


ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
inputs = tokenizer.encode("Once upon a time ", return_tensors="pt")
outputs = model.generate(inputs)
tokenizer.decode(outputs[0])